In [1]:
!pip install --upgrade pip
!pip install torch transformers accelerate bitsandbytes peft trl datasets huggingface_hub
!pip install -q einops

In [2]:
!pip install --upgrade multiprocess dill

  Using cached multiprocess-0.70.18-py310-none-any.whl.metadata (7.5 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
Using cached multiprocess-0.70.18-py310-none-any.whl (134 kB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [multiprocess] [multiprocess]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.4.0 which is incompatible.
datasets 4.0.0 requires multiprocess<0.70.17, but you have multiprocess 0.70.18 which

In [3]:
!pip install --upgrade datasets


  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling multiprocess-0.70.18:
      Successfully uninstalled multiprocess-0.70.18
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [multiprocess] [multiprocess]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.4 requires dill>=0.4.0, but you have dill 0.3.8 which is incompatible.
pathos 0.3.4 requires multiprocess>=0.70.18, but you have multiprocess 0.70.16 which is incomp

In [2]:
!pip install --upgrade datasets


In [3]:
from huggingface_hub import notebook_login

# Run this and enter your Hugging Face token when prompted
notebook_login()

In [4]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from datasets import load_dataset
import os

In [5]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

# 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Required for batched training

# Load pre-trained model with QLoRA (4-bit)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically uses GPU(s)
    trust_remote_code=True
)

# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
peft_config = LoraConfig(
    r=64,  # Rank of the low-rank matrices
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Common for Llama/Mistral
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap the model with LoRA
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 18,350,080 || all params: 3,231,099,904 || trainable%: 0.5679


In [9]:
# Load dataset (example: Dolly-15k)
dataset = load_dataset("json", data_files="train2.jsonl", split="train")

# Optional: Use a smaller subset for faster testing
dataset = dataset.select(range(1000))  # Remove this line for full training

def format_instruction(example):
    return {
        "text": f"<|start_of_turn|>user\n{example['Instruction']}<|end_of_turn|>"
               f"<|start_of_turn|>model\n{example['Response']}<|end_of_turn|>"
    }

dataset = dataset.map(format_instruction, remove_columns=dataset.column_names)

In [10]:
!pip install --upgrade transformers

In [11]:
!pip show transformers

Name: transformers
Version: 4.55.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl


In [12]:
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,       # Reduce if OOM
    gradient_accumulation_steps=8,       # Effective batch size = 8
    optim="paged_adamw_8bit",            # Handle memory better
    save_steps=100,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,
    bf16=False,  # Enable if GPU supports it (Ampere+)
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",  # Change to "wandb" if using logging
    save_strategy="steps",
    save_total_limit=2,
    load_best_model_at_end=False,
    eval_strategy="no",  # No eval set for simplicity
    gradient_checkpointing=True,
    run_name="q_lora_finetune_run"
)

In [13]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Not masked LM, causal LM
)

In [14]:
!pip install --upgrade trl peft transformers accelerate bitsandbytes

In [15]:
def formatting_func(example):
    text = example["text"]
    return [text]

In [16]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],           # replace "text" with your column name
        truncation=True,
        max_length=512,
        padding=False,
        return_attention_mask=True
    )

In [17]:
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=2,
    remove_columns=["text"],  # Remove original text column
    desc="Tokenizing dataset"
)

In [18]:
!pip install --upgrade trl peft transformers accelerate bitsandbytes

In [19]:
import trl
print("trl version:", trl.__version__)

trl version: 0.21.0


In [34]:
!pip uninstall -y trl

Found existing installation: trl 0.21.0
Uninstalling trl-0.21.0:
  Successfully uninstalled trl-0.21.0


In [35]:
!pip install trl==0.21.0

  Using cached trl-0.21.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.21.0-py3-none-any.whl (511 kB)


In [22]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,  # Function to merge Instruction + Response
)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'meta-llama/Llama-3.2-3B-Instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:687: UserWarning:

In [23]:
if hasattr(model, "peft_config"):
    model = model.unload()

In [24]:
model = get_peft_model(model, peft_config)

In [25]:
trainer.train()
trainer.save_model("./q_lora_adapter")
tokenizer.save_pretrained("./q_lora_adapter")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
25,1.977100
50,1.500100
75,1.421200
100,1.296300
125,1.407300
150,1.291100
175,1.191300
200,1.222300
225,1.257600
250,1.280000


('./q_lora_adapter/tokenizer_config.json',
 './q_lora_adapter/special_tokens_map.json',
 './q_lora_adapter/chat_template.jinja',
 './q_lora_adapter/tokenizer.json')

In [26]:
model.save_pretrained("./q_lora_adapter")
tokenizer.save_pretrained("./q_lora_adapter")
print("✅ Adapter saved!")

✅ Adapter saved!


In [34]:
def generate(model, prompt, max_new_tokens=150):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [37]:
# Load datasets
ds1 = load_dataset("json", data_files="train2.jsonl", split="train")
ds2 = load_dataset("json", data_files="constitution_qa.jsonl", split="train")
ds3 = load_dataset("json", data_files="ipc_qa.jsonl", split="train")
ds4 = load_dataset("json", data_files="crpc_qa.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [38]:
def standardize_example(example):
    # Handle different field names
    if "question" in example:
        instruction = example["question"]
    elif "Instruction" in example:
        instruction = example["Instruction"]
    else:
        instruction = example["instruction"]

    if "answer" in example:
        response = example["answer"]
    elif "Response" in example:
        response = example["Response"]
    else:
        response = example["response"]

    # Optional: include context if available
    context = ""
    if "context" in example and example["context"].strip():
        context = f"\n\nContext: {example['context'].strip()}"

    # Return unified format
    return {
        "instruction": instruction.strip(),
        "context": context,
        "response": response.strip()
    }

# Apply to all datasets
ds1_std = ds1.map(lambda x: standardize_example(x), remove_columns=ds1.column_names)
ds2_std = ds2.map(lambda x: standardize_example(x), remove_columns=ds2.column_names)
ds3_std = ds3.map(lambda x: standardize_example(x), remove_columns=ds3.column_names)
ds4_std = ds4.map(lambda x: standardize_example(x), remove_columns=ds4.column_names)

Map:   0%|          | 0/24607 [00:00<?, ? examples/s]

Map:   0%|          | 0/4082 [00:00<?, ? examples/s]

Map:   0%|          | 0/2267 [00:00<?, ? examples/s]

Map:   0%|          | 0/8194 [00:00<?, ? examples/s]

In [40]:
from datasets import concatenate_datasets

merged_dataset = concatenate_datasets([ds1_std, ds2_std, ds3_std, ds4_std])

print(f"Total training examples: {len(merged_dataset)}")

Total training examples: 39150


In [41]:
def format_instruction(example):
    return {
        "text": (
            f"### Instruction:\n{example['instruction']}"
            f"{example['context']}\n\n"
            f"### Response:\n{example['response']}"
        )
    }

# Apply formatting
merged_dataset = merged_dataset.map(format_instruction, remove_columns=merged_dataset.column_names)

Map:   0%|          | 0/39150 [00:00<?, ? examples/s]

In [45]:
# Load your previously saved adapter
model = PeftModel.from_pretrained(base_model, "./q_lora_adapter")

# No need to re-wrap with get_peft_model() — already a PeftModel
# Just continue training
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,  # Function to merge Instruction + Response
)

# This will update the existing LoRA weights
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'meta-llama/Llama-3.2-3B-Instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:687: UserWarning:

Step,Training Loss
25,1.978300
50,1.501400
75,1.421200
100,1.296500
125,1.407600
150,1.291200
175,1.192000
200,1.222400
225,1.257600
250,1.281000


TrainOutput(global_step=375, training_loss=1.3151659800211588, metrics={'train_runtime': 1059.5836, 'train_samples_per_second': 2.831, 'train_steps_per_second': 0.354, 'total_flos': 1.264105660916736e+16, 'train_loss': 1.3151659800211588})

In [46]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Your model name
model_name = "meta-llama/Llama-3.2-3B-Instruct"

# 4-bit config (same as before)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load base model (no LoRA)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
# Now load your adapter on top
from peft import PeftModel

finetuned_model = PeftModel.from_pretrained(
    base_model, 
    "./q_lora_adapter"  # Your trained LoRA
)

In [48]:
def ask(model, prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):].strip()

In [49]:
questions = [
    "What is the difference between murder and culpable homicide?",
    "Explain the concept of 'basic structure doctrine' in Indian Constitution.",
    "When can a writ of habeas corpus be filed?",
    "What are the rights of an accused under Article 22?",
    "What is anticipatory bail under Section 438 of CrPC?"
]

for q in questions:
    prompt = f"### Instruction:\n{q}\n\n### Response:\n"
    
    print(f"🔸 Question: {q}\n")
    
    print("🟡 Base Model:")
    print(ask(base_model, prompt))
    print("\n")
    
    print("🟢 Fine-Tuned Model:")
    print(ask(finetuned_model, prompt))
    print("\n" + "="*80 + "\n")

🔸 Question: What is the difference between murder and culpable homicide?

🟡 Base Model:
Murder is the intentional and premeditated killing of another person, whereas culpable homicide is the killing of another person without any intention or motive, but with a reckless disregard for life. In India, the Indian Penal Code (IPC) 304A deals with the punishment for culpable homicide not amounting to murder. The main difference between the two is that murder is a more serious offence and is punishable with the death penalty or imprisonment for life. Culpable homicide, on the other hand, is punishable with imprisonment for a specified term.

### Example:
A man intentionally kills another man with a sharp object, and the killing is done in a premeditated manner. In this case, it would be murder. However, if the man kills the other man in a fit of rage without any intention or motive, and the killing is done recklessly without any regard for life, it would be culpable homicide.

### Instruction